# Analiza wykorzystania różnych heurystyk przy rozwiązaniu problemu komiwojażera

Grupa: Amelia Madej, Justyna Sarkowicz, Olga Sieradzan, Weronika Duda i Aleksandra Węgrzyn

## Biblioteki

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
import random
import math
from collections import deque
 

## Pobór danych 

In [2]:
# file_path1 = "C:/Users/olgas/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_48.xlsx"
# file_path2 = "C:/Users/olgas/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_76.xlsx"
# file_path3 = "C:/Users/olgas/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_127.xlsx"

# file_path1 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_48.xlsx"
# file_path2 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_76.xlsx"
# file_path3 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_127.xlsx"

# file_path1 = "C:/Users/ameli/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_48.xlsx"
# file_path2 = "C:/Users/ameli/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_76.xlsx"
# file_path3 = "C:/Users/ameli/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_127.xlsx"

# file_path1 = "C:/Users/wera6/Downloads/Dane_TSP_48.xlsx"
# file_path2 = "C:/Users/wera6/Downloads/Dane_TSP_76.xlsx"
# file_path3 = "C:/Users/wera6/Downloads/Dane_TSP_127.xlsx"

# index_col=0 zamienia pierwszą kolumne na indeksy wierszy 
# .to_numpy() zamienia ramkę danych na macierz
data1 = pd.read_excel(file_path1, index_col=0).to_numpy()

data2 = pd.read_excel(file_path2, index_col=0).to_numpy()

data3 = pd.read_excel(file_path3, index_col=0).to_numpy()

## Rodzaje ruchów

In [3]:
# Wybieramy dwie pozycje i zmieniamy je miejscami 
def swap_move(path):  
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:]
        new_path[i], new_path[j] = new_path[j], new_path[i]
        yield new_path
 

# Odwracamy segment pomiędzy wybranymi wartościami czyli 
# [a b c d e] -> [a d c b e]
def two_opt_move(path):
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:i] + path[i:j][::-1] + path[j:]
        yield new_path
 
# Wybieramy jeden wierzchołek w ścieżce, usuwamy z jednego miejsca i przerzucamy w inne miejsce
def insertion_move(path):
    n = len(path)
    for i in range(n):
        for j in range(n):
            if i != j:
                new_path = path[:]
                node = new_path.pop(i)
                new_path.insert(j, node)
                yield new_path


## Długość ścieżki

In [4]:
def calculate_path_cost(matrix, path):
    return sum(matrix[path[i - 1]][path[i]] for i in range(len(path))) + matrix[path[-1]][path[0]]


## Algorytm najbliższego sąsiada NN

In [5]:

# Klasyczny algorytm NN

def nearest_neighbor(matrix, start_point=0, max_time=60, random_restart=1):
    n = len(matrix)
    best_path, best_cost = None, float('inf')
    start_time = time.time()

    for _ in range(random_restart):
        # Wybór losowego punktu startowego przy wielu restartach
        current_start_point = np.random.randint(0, n) if random_restart > 1 else start_point

        # Przygotowanie zmiennych dla algorytmu NN
        visited = [False] * n
        path = [current_start_point]
        visited[current_start_point] = True

        # Generowanie ścieżki metodą najbliższego sąsiada
        while len(path) < n:
            last = path[-1]
            next_city = np.argmin([
                matrix[last][j] if not visited[j] else np.inf
                for j in range(n)
            ])
            path.append(next_city)
            visited[next_city] = True

        # Oblicz koszt uzyskanej ścieżki
        cost = calculate_path_cost(matrix, path)

        # Sprawdź, czy znaleziono lepsze rozwiązanie
        if cost < best_cost:
            best_path, best_cost = path, cost

        # Przerwij, jeśli przekroczono limit czasu
        if time.time() - start_time > max_time:
            break

    return best_path, best_cost




# Policzenie łącznej odległości między startem a końcem                



# Optymalizacja rozwiązania 
# Argumenty: matrix odległości / dotychczasowe rozwiązanie / ['swap', '2-opt', 'insertion'] / maksymalna liczba iteracji
def local_search(matrix, initial_path, move_generators, max_iterations):
    best_path = initial_path
    best_cost = calculate_path_cost(matrix, best_path)
 
    iteration = 0
    improved = True
    while improved and iteration < max_iterations:
        improved = False
        for move_generator in move_generators:
            for new_path in move_generator(best_path):
                new_cost = calculate_path_cost(matrix, new_path)
                if new_cost < best_cost:
                    best_path, best_cost = new_path, new_cost
                    improved = True
                    break  
            if improved:
                break
        iteration += 1
 
    return best_path, best_cost

# Ostateczna funkcja 

#  Parametry:
# file_path (str): Ścieżka do pliku Excel zawierającego macierz odległości.
# start_point (int): Miasto początkowe dla algorytmu.
# max_time (int): Maksymalny czas dozwolony na obliczenia (w sekundach).
# move_types (list): Lista typów ruchów dla wyszukiwania lokalnego (np. ['swap', '2-opt', 'insertion']).
# random_restart (int): Liczba losowych restartów w celu wyjścia poza lokalne minima, 
#                       przy każdej iteracji wybieramy nowy start_point a nie cały czas ten sam co ustawiony, jeżeli  random_restart > 1
# max_iterations (int): Maksymalna liczba iteracji dla wyszukiwania lokalnego.
 
# Zwraca:
# best_path (list): Znaleziono optymalną ścieżkę.
# best_cost (float): Długość optymalnej ścieżki.

# Jeżeli move_types=None to używamy wszytskich sposobów do optymalizacji

def solve_tsp(matrix, start_point=0, max_time=60, move_types=None, random_restart=1, max_iterations=100):
   
    
    move_generators = []
    if move_types is None:
        move_types = ['swap', '2-opt', 'insertion']  
    if 'swap' in move_types:
        move_generators.append(swap_move)
    if '2-opt' in move_types:
        move_generators.append(two_opt_move)
    if 'insertion' in move_types:
        move_generators.append(insertion_move)
 
    best_path, best_cost = None, float('inf')
    start_time = time.time()

    # Iteracje z różnym startem
    for _ in range(random_restart):
        # wybranie punktu startującego 
        initial_start_point = np.random.randint(0, len(matrix)) if random_restart > 1 else start_point
 
        # Początkowo generujemy rożwiązanie za pomocą klasycznego NN
        initial_path, initial_cost = nearest_neighbor(matrix, start_point=initial_start_point)
 
        #print(f"Initial NN path (start {initial_start_point}): {initial_path} with cost: {initial_cost}")
 
        # Potem przeprowadzamy optymalizacje z pomocą innych parametrów
        current_path, current_cost = local_search(matrix, initial_path, move_generators, max_iterations)
 
        # Aktualizacja najlepszego wyniku
        if current_cost < best_cost:
            best_path, best_cost = current_path, current_cost
        
 
        # Zabezpieczenie przed przekroczeniem czasu 
        if time.time() - start_time > max_time:
            break
 
    return best_path, best_cost
 

# przykład użycia 
best_path, best_cost = solve_tsp(
    data1, 
    start_point=0, 
    max_time=60, 
    move_types='swap', 
    random_restart=1, 
    max_iterations=150
)


### Generowanie rozwiązań 

Za podstawowe dane przyjmujemy :

* start_ point = 0 

* max_time = 500

* random_restart = 1 

* max_itaration = 150 

Nastepnie badane są wpływy poszczególnych parametrów na każdą metodę 

In [6]:
# Klasyczna metoda NN 
classic_p_1, classic_c_1 = nearest_neighbor(data1, start_point=0 , max_time=500, random_restart=1)


In [7]:
classic_p_2, classic_c_2 = nearest_neighbor(data2, start_point=0, max_time=500, random_restart=1)


In [8]:
classic_p_3, classic_c_3 = nearest_neighbor(data3, start_point=0, max_time=500, random_restart=1)

In [9]:
# Move type = 'swap'

swap_p_1, swap_c_1 = solve_tsp(
    data1, 
    start_point=0, 
    max_time=500, 
    move_types='swap', 
    random_restart=1, 
    max_iterations=150
)



In [10]:
swap_p_2, swap_c_2 = solve_tsp(
    data2, 
    start_point=0, 
    max_time=500, 
    move_types='swap', 
    random_restart=1, 
    max_iterations=150
)



In [11]:
swap_p_3, swap_c_3 = solve_tsp(
    data3, 
    start_point=0, 
    max_time=500, 
    move_types='swap', 
    random_restart=1, 
    max_iterations=150
)

In [12]:
# Move typ = "2 opt"

opt_p_1, opt_c_1 = solve_tsp(
    data1, 
    start_point=0, 
    max_time=500, 
    move_types= '2-opt', 
    random_restart=1, 
    max_iterations=150
)



In [13]:
opt_p_2, opt_c_2 = solve_tsp(
    data2, 
    start_point=0, 
    max_time=500, 
    move_types= '2-opt', 
    random_restart=1, 
    max_iterations=150
)



In [14]:
opt_p_3, opt_c_3 = solve_tsp(
    data3, 
    start_point=0, 
    max_time=500, 
    move_types= '2-opt', 
    random_restart=1, 
    max_iterations=150
)

In [15]:
# Move typ = "insertion"

ins_p_1, ins_c_1 = solve_tsp(
    data1, 
    start_point=0, 
    max_time=500, 
    move_types= 'insertion', 
    random_restart=1, 
    max_iterations=150
)



In [16]:
ins_p_2, ins_c_2 = solve_tsp(
    data2, 
    start_point=0, 
    max_time=500, 
    move_types= 'insertion', 
    random_restart=1, 
    max_iterations=150
)



In [17]:
ins_p_3, ins_c_3 = solve_tsp(
    data3, 
    start_point=0, 
    max_time=500, 
    move_types= 'insertion', 
    random_restart=1, 
    max_iterations=150
)

Zebranie wyników z podstawowymi parametrami 

In [18]:
W1 =  {
    "Długość ścieżki": [classic_c_1, swap_c_1, opt_c_1, ins_c_1]
}

c1 = pd.DataFrame(data = W1)
c1.index = ["Klasyczna","Swap" , "2-opt", "Insertion"]

W2 =  {
    "Długość ścieżki": [classic_c_2, swap_c_2, opt_c_2, ins_c_2]
}

c2 = pd.DataFrame(data = W2)
c2.index = ["Klasyczna","Swap" , "2-opt", "Insertion"]

W3 =  {
    "Długość ścieżki": [classic_c_3, swap_c_3, opt_c_3, ins_c_3]
}

c3 = pd.DataFrame(data = W3)
c3.index = ["Klasyczna","Swap" , "2-opt", "Insertion"]

Badanie wpływu parametru MAX ITERATIONS

In [20]:
# Stałe parametry
start_point = 0
max_time = 500
random_restart = 1

# DANE NR 1 
results = []
for max_iterations in range(1, 301):
    _, best_cost = solve_tsp(data1, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost})


itterration1 = pd.DataFrame(results)


# DANE NR 2
results = []
for max_iterations in range(1, 301):
    _, best_cost = solve_tsp(data2, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost})

# Tworzenie ramki danych
itterration2 = pd.DataFrame(results)


# DANE NR 3
results = []
for max_iterations in range(1, 301):
    _, best_cost = solve_tsp(data3, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost})

# Tworzenie ramki danych
itterration3 = pd.DataFrame(results)

Badanie wpływu parametru random restart

In [21]:
# Stałe parametry
start_point = 0
max_time = 500
max_iterations = 150

# DANE NR 1 
results = []
for random_restart in range(1, 31):
    _, best_cost = solve_tsp(data1, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": random_restart, "WYNIK": best_cost})


restar1 = pd.DataFrame(results)


# DANE NR 2
results = []
for random_restart in range(1, 31):
    _, best_cost = solve_tsp(data2, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": random_restart, "WYNIK": best_cost})

# Tworzenie ramki danych
restart2 = pd.DataFrame(results)


# DANE NR 3
results = []
for random_restart in range(1, 31):
    _, best_cost = solve_tsp(data3, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": random_restart, "WYNIK": best_cost})

# Tworzenie ramki danych
restart3 = pd.DataFrame(results)

Wystartowanie z 4 różnych losowych puntktów startowych

In [22]:
# Stałe parametry
start_points = [np.random.randint(0, len(data1)), np.random.randint(0, len(data1)), np.random.randint(0, len(data1)), np.random.randint(0, len(data1))]
max_time = 500
max_iterations = 150
random_restart = 1

# DANE NR 1 
results = []
for start_point in start_points:
    _, best_cost = solve_tsp(data1, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": start_point, "WYNIK": best_cost})


point1 = pd.DataFrame(results)

start_points = [np.random.randint(0, len(data2)), np.random.randint(0, len(data2)), np.random.randint(0, len(data2)), np.random.randint(0, len(data2))]

# DANE NR 2
results = []
for start_point in start_points:
    _, best_cost = solve_tsp(data2, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": start_point, "WYNIK": best_cost})

# Tworzenie ramki danych
point2 = pd.DataFrame(results)

start_points = [np.random.randint(0, len(data3)), np.random.randint(0, len(data3)), np.random.randint(0, len(data3)), np.random.randint(0, len(data3))]
# DANE NR 3
results = []
for start_point in start_points:
    _, best_cost = solve_tsp(data3, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": start_point, "WYNIK": best_cost})

# Tworzenie ramki danych
point3 = pd.DataFrame(results)

Zapis wyników w pliku EXCEL

In [23]:
res = {
    "Porównanie_metod+1": c1,
    "max_iterations_1": itterration1,
    "Restart_1": restar1,
    "Point_1" : point1,
    "Porównanie_metod_2": c2,
    "max_iterations_2": itterration2,
    "Restart_2": restart2,
    "Point_2" : point2,
    "Porównanie_metod_3": c3,
    "max_iterations_3": itterration3,
    "Restart_3": restart3,
    "Point_3" : point3,
    
}

# Ścieżka do pliku Excel
file_name = "NN.xlsx"

# Zapisujemy dane do Excela
with pd.ExcelWriter(file_name) as writer:
    for sheet_name, df in res.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Wyniki zostały zapisane w pliku {file_name}")

Wyniki zostały zapisane w pliku NN.xlsx


## Algorytm wspinaczki z multistartem (IHC)

In [24]:
def local_search(route, distance_matrix, neighborhood_type="swap"):
  

    # Mapowanie typów sąsiedztwa na funkcje
    neighborhood_functions = {
        "swap": swap_move,
        "two_opt": two_opt_move,
        "insertion": insertion_move
    }
    
    if neighborhood_type not in neighborhood_functions:
        raise ValueError(f"Nieznany typ sąsiedztwa: {neighborhood_type}")

    neighbor_func = neighborhood_functions[neighborhood_type]

    n = len(route)
    best_distance = calculate_path_cost( distance_matrix, route)
    improved = True

    while improved:
        improved = False
        for neighbor in neighbor_func(route):
            new_distance = calculate_path_cost( distance_matrix, neighbor)
            if new_distance < best_distance:
                route = neighbor
                best_distance = new_distance
                improved = True
                break  # Skok do następnego sąsiedztwa

    return route, best_distance



    # Algorytm wspinaczki z multistartem dla problemu TSP z kryterium czasowym.
    # Jeśli neighborhood_type=None, działa jak klasyczny IHC bez optymalizacji.
    # distance_matrix: Macierz odległości między miastami.
    # num_starts: Maksymalna liczba startów (iteracji).
    # neighborhood_type: Typ sąsiedztwa ("swap", "two_opt", "insertion").
    # time_limit: Maksymalny czas działania algorytmu w sekundach.
   
def iterated_hill_climbing(distance_matrix, num_starts=100, neighborhood_type="swap", time_limit=None):
  
    n = len(distance_matrix)
    best_route = None
    best_distance = float('inf')

    start_time = time.time()  # Zapisz czas rozpoczęcia

    for start in range(num_starts):
        # Sprawdzenie limitu czasu
        if time_limit and (time.time() - start_time > time_limit):
            print(f"Przekroczono limit czasu ({time_limit} s).")
            break

        # Losowe rozwiązanie początkowe
        initial_route = list(range(n))
        random.shuffle(initial_route)

        # Lokalna wspinaczka lub brak optymalizacji
        local_route, local_distance = local_search(
            initial_route, distance_matrix, neighborhood_type
        )

        # Aktualizacja najlepszego rozwiązania
        if local_distance < best_distance:
            best_route = local_route
            best_distance = local_distance

    return best_route, best_distance

### Generowanie wyników 

Za podstawowe dane przyjmujemy :

* time_limit = 1000

* num_starts = 10 000 

Nastepnie badane są wpływy poszczególnych parametrów 

In [52]:
# Metoda SWAP
swap_p_1, swap_c_1 = iterated_hill_climbing(data1, num_starts=150,neighborhood_type= "swap", time_limit=1000)

In [53]:
swap_p_2, swap_c_2 = iterated_hill_climbing(data2, num_starts=150,neighborhood_type= "swap", time_limit=1000)

In [54]:
swap_p_3, swap_c_3 = iterated_hill_climbing(data3, num_starts=150,neighborhood_type= "swap", time_limit=1000)

Przekroczono limit czasu (1000 s).


In [55]:
# Metoda Opt-2
opt_p_1, opt_c_1 = iterated_hill_climbing(data1, num_starts=150,neighborhood_type= "two_opt", time_limit=1000)

In [56]:
opt_p_2, opt_c_2 = iterated_hill_climbing(data2, num_starts=150,neighborhood_type= "two_opt", time_limit=1000)

In [57]:
opt_p_3, opt_c_3 = iterated_hill_climbing(data3, num_starts=150,neighborhood_type= "two_opt", time_limit=1000)

KeyboardInterrupt: 

In [31]:
# Metoda insertion

ins_p_1, ins_c_1 = iterated_hill_climbing(data1, num_starts=150,neighborhood_type= "insertion", time_limit=1000)

In [32]:
ins_p_2, ins_c_2 = iterated_hill_climbing(data2, num_starts=150,neighborhood_type= "insertion", time_limit=1000)

In [33]:

ins_p_3, ins_c_3 = iterated_hill_climbing(data3, num_starts=150, neighborhood_type= "insertion", time_limit=1000)

Przekroczono limit czasu (1000 s).


In [ ]:
W9 = {
    "Swap" : [swap_c_1, swap_c_2, swap_c_3],
    "Opt": [opt_c_1, opt_c_2, opt_c_3],
    "Insertion" : [ins_c_1, ins_c_2, ins_c_3]
}

metody = pd.DataFrame(W9)

Badanie wpływu parametru num_starts 

In [34]:
# Stałe parametry
time_limit = 1000

# DANE NR 1 
results = []
for num_starts in range(1, 200,  20):
    _, best_cost = iterated_hill_climbing(data1, num_starts=num_starts ,neighborhood_type= "two_opt", time_limit=time_limit)
    results.append({"PARAMETR": num_starts, "WYNIK": best_cost})


itterration1 = pd.DataFrame(results)


# DANE NR 2

results = []
for num_starts in range(1, 200,  20):
    _, best_cost = iterated_hill_climbing(data2, num_starts=num_starts ,neighborhood_type= "two_opt", time_limit=time_limit)
    results.append({"PARAMETR": num_starts, "WYNIK": best_cost})

# Tworzenie ramki danych
itterration2 = pd.DataFrame(results)


# DANE NR 3

results = []
for num_starts in range(1, 200,  20):
    _, best_cost = iterated_hill_climbing(data3, num_starts=num_starts ,neighborhood_type= "two_opt", time_limit=time_limit)
    results.append({"PARAMETR": num_starts, "WYNIK": best_cost})

# Tworzenie ramki danych
itterration3 = pd.DataFrame(results)

Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).


In [50]:
merged_itt = itterration1.merge(itterration2, on="PARAMETR").merge(itterration3, on="PARAMETR")

Badanie wpływu parametru time_limit

In [35]:
# # Stałe parametry
# num_starts = 1000000

# # DANE NR 1 
# results = []
# for time_limit in range(0, 1000, 100):
#     _, best_cost = iterated_hill_climbing(data1, num_starts=num_starts ,neighborhood_type= "two_opt", time_limit=time_limit)
#     results.append({"PARAMETR": time_limit, "WYNIK": best_cost})


# time1 = pd.DataFrame(results)


# # DANE NR 2

# results = []
# for time_limit in range(0, 1000,  100):
#     _, best_cost = iterated_hill_climbing(data2, num_starts=num_starts ,neighborhood_type= "two_opt", time_limit=time_limit)
#     results.append({"PARAMETR": time_limit, "WYNIK": best_cost})

# # Tworzenie ramki danych
# time2 = pd.DataFrame(results)


# # DANE NR 3

# results = []
# for time_limit in range(0, 1000, 100):
#     _, best_cost = iterated_hill_climbing(data3, num_starts=num_starts ,neighborhood_type= "two_opt", time_limit=time_limit)
#     results.append({"PARAMETR": time_limit, "WYNIK": best_cost})

# # Tworzenie ramki danych
# time3 = pd.DataFrame(results)



In [51]:
resu = {
    "Porównanie_metod": metody,
    "max_iterations": merged_itt   
}

# Ścieżka do pliku Excel
file_name = "IHC.xlsx"

# Zapisujemy dane do Excela
with pd.ExcelWriter(file_name) as writer:
    for sheet_name, df in resu.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Wyniki zostały zapisane w pliku {file_name}")

Wyniki zostały zapisane w pliku IHC.xlsx


## Algorytm symulowanego wyżarzania (SA)

In [65]:

 
    # Algorytm wyżarzania dla problemu komiwojażera (TSP).

    
    # :param distance_matrix: Macierz odległości między miastami.
    # :param initial_temperature: Temperatura początkowa.

# To początkowa wartość temperatury, która kontroluje, jak duża 
# jest początkowa tolerancja algorytmu na akceptowanie gorszych 
# rozwiązań. Wysoka temperatura pozwala eksplorować więcej rozwiązań,
# nawet tych o wyższych kosztach, unikając zbyt wczesnego utknięcia w minimum lokalnym.

    # :param cooling_rate: Współczynnik chłodzenia (wartość w zakresie (0, 1)).

# To współczynnik określający, jak szybko temperatura spada
# w każdej iteracji. Jest liczbą z zakresu (0, 1). Im mniejszy 
# cooling_rate, tym szybciej temperatura spada, co może prowadzić
# do wcześniejszego zatrzymania algorytmu, ograniczając eksplorację przestrzeni rozwiązań.

    # :param max_iterations: Maksymalna liczba iteracji.
    # :param min_temperature: Minimalna temperatura.

# To próg, przy którym algorytm przestaje obniżać temperaturę
# i kończy działanie. Oznacza, że eksploracja kończy się, gdy
# temperatura spada poniżej tej wartości.

    # :param time_limit: Maksymalny czas działania algorytmu w sekundach.
    # :param move_type: Typ ruchu do generowania sąsiada ("swap", "insertion", "two_opt").
    # :return: Najlepsze rozwiązanie i jego koszt.




IndentationError: expected an indented block after function definition on line 30 (3752839163.py, line 39)

In [88]:
def simulated_annealing(
    distance_matrix,
    initial_temperature=1000,
    cooling_rate=0.95,
    max_iterations=1000,
    min_temperature=1e-3,
    time_limit=None,
    move_type="swap"  
):
    n = len(distance_matrix)
    initial_route = list(range(n))
    random.shuffle(initial_route)
    
    # Mapowanie typu ruchu na odpowiednią funkcję generatora sąsiadów
    move_generators = {
        "swap": swap_move,
        "two_opt": two_opt_move,
        "insertion": insertion_move
    }

    if move_type not in move_generators:
        raise ValueError(f"Nieznany typ ruchu: {move_type}")

    move_generator = move_generators[move_type]

    current_solution = initial_route
    current_cost = calculate_path_cost(distance_matrix, current_solution)
    best_solution = current_solution[:]
    best_cost = current_cost

    temperature = initial_temperature
    start_time = time.time()

    for iteration in range(max_iterations):
        if time_limit and (time.time() - start_time > time_limit):
            print("Przekroczono limit czasu.")
            break

        if temperature < min_temperature:
            print("Temperatura osiągnęła wartość minimalną.")
            break

        # Generowanie sąsiada na podstawie wybranego generatora ruchów
        neighbors = list(move_generator(current_solution))
        neighbor = random.choice(neighbors)
        neighbor_cost = calculate_path_cost(distance_matrix, neighbor)

        delta = neighbor_cost - current_cost
        acceptance_probability = math.exp(-delta / temperature) if delta > 0 else 1

        if random.random() < acceptance_probability:
            current_solution = neighbor
            current_cost = neighbor_cost

        if current_cost < best_cost:
            best_solution = current_solution[:]
            best_cost = current_cost

        temperature *= cooling_rate

        if iteration % 100 == 0:
            print(f"Iteracja {iteration}, najlepszy koszt: {best_cost}, temperatura: {temperature:.2f}")

    return best_solution, best_cost


### Generowanie wyników

Za podstawowe dane przyjmujemy :

* initial_temperature= 10000,

* cooling_rate= 0.95,

* max_iterations= 10000,

* min_temperature= 0.00001,

Nastepnie badane są wpływy poszczególnych parametrów 

In [89]:
swap_p_1, swap_c_1= simulated_annealing(
        data1,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "swap"
    )

Iteracja 0, najlepszy koszt: 50773, temperatura: 9900.00
Iteracja 100, najlepszy koszt: 45151, temperatura: 3623.72
Iteracja 200, najlepszy koszt: 42139, temperatura: 1326.40
Iteracja 300, najlepszy koszt: 33232, temperatura: 485.50
Iteracja 400, najlepszy koszt: 32709, temperatura: 177.71
Iteracja 500, najlepszy koszt: 28270, temperatura: 65.05
Iteracja 600, najlepszy koszt: 22138, temperatura: 23.81
Iteracja 700, najlepszy koszt: 20636, temperatura: 8.72
Iteracja 800, najlepszy koszt: 20483, temperatura: 3.19
Iteracja 900, najlepszy koszt: 20094, temperatura: 1.17
Iteracja 1000, najlepszy koszt: 19718, temperatura: 0.43
Iteracja 1100, najlepszy koszt: 19136, temperatura: 0.16
Iteracja 1200, najlepszy koszt: 18942, temperatura: 0.06
Iteracja 1300, najlepszy koszt: 18930, temperatura: 0.02
Iteracja 1400, najlepszy koszt: 18827, temperatura: 0.01
Iteracja 1500, najlepszy koszt: 18827, temperatura: 0.00
Iteracja 1600, najlepszy koszt: 18765, temperatura: 0.00
Iteracja 1700, najlepszy kos

In [90]:
swap_p_2, swap_c_2= simulated_annealing(
        data2,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "swap"
    )

Iteracja 0, najlepszy koszt: 606012.4615383524, temperatura: 9900.00
Iteracja 100, najlepszy koszt: 468676.5122427727, temperatura: 3623.72
Iteracja 200, najlepszy koszt: 413161.28589745396, temperatura: 1326.40
Iteracja 300, najlepszy koszt: 386208.37265016144, temperatura: 485.50
Iteracja 400, najlepszy koszt: 342853.5666960335, temperatura: 177.71
Iteracja 500, najlepszy koszt: 315086.0290880598, temperatura: 65.05
Iteracja 600, najlepszy koszt: 303736.84745414037, temperatura: 23.81
Iteracja 700, najlepszy koszt: 297326.47341677954, temperatura: 8.72
Iteracja 800, najlepszy koszt: 287814.4290794217, temperatura: 3.19
Iteracja 900, najlepszy koszt: 280260.92829711584, temperatura: 1.17
Iteracja 1000, najlepszy koszt: 277977.2902768642, temperatura: 0.43
Iteracja 1100, najlepszy koszt: 273885.20933472825, temperatura: 0.16
Iteracja 1200, najlepszy koszt: 271232.2515108877, temperatura: 0.06
Iteracja 1300, najlepszy koszt: 269283.2015805077, temperatura: 0.02
Iteracja 1400, najlepszy 

In [91]:
swap_p_3, swap_c_3= simulated_annealing(
        data3,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "swap"
    )

Iteracja 0, najlepszy koszt: 634368.7713949656, temperatura: 9900.00
Iteracja 100, najlepszy koszt: 570920.4566910813, temperatura: 3623.72
Iteracja 200, najlepszy koszt: 512675.41667674796, temperatura: 1326.40
Iteracja 300, najlepszy koszt: 432840.33802583703, temperatura: 485.50
Iteracja 400, najlepszy koszt: 410330.5516985676, temperatura: 177.71
Iteracja 500, najlepszy koszt: 399782.9492674563, temperatura: 65.05
Iteracja 600, najlepszy koszt: 395427.1525018052, temperatura: 23.81
Iteracja 700, najlepszy koszt: 386056.60848754604, temperatura: 8.72
Iteracja 800, najlepszy koszt: 372771.03328075405, temperatura: 3.19
Iteracja 900, najlepszy koszt: 371472.50138322904, temperatura: 1.17
Iteracja 1000, najlepszy koszt: 359366.65890511795, temperatura: 0.43
Iteracja 1100, najlepszy koszt: 345164.1908900171, temperatura: 0.16
Iteracja 1200, najlepszy koszt: 335715.4476852102, temperatura: 0.06
Iteracja 1300, najlepszy koszt: 323062.24403701, temperatura: 0.02
Iteracja 1400, najlepszy ko

In [92]:
opt_p_1, opt_c_1= simulated_annealing(
        data1,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )

Iteracja 0, najlepszy koszt: 48896, temperatura: 9900.00
Iteracja 100, najlepszy koszt: 43121, temperatura: 3623.72
Iteracja 200, najlepszy koszt: 41391, temperatura: 1326.40
Iteracja 300, najlepszy koszt: 34680, temperatura: 485.50
Iteracja 400, najlepszy koszt: 27004, temperatura: 177.71
Iteracja 500, najlepszy koszt: 22304, temperatura: 65.05
Iteracja 600, najlepszy koszt: 20126, temperatura: 23.81
Iteracja 700, najlepszy koszt: 18980, temperatura: 8.72
Iteracja 800, najlepszy koszt: 17981, temperatura: 3.19
Iteracja 900, najlepszy koszt: 16028, temperatura: 1.17
Iteracja 1000, najlepszy koszt: 15638, temperatura: 0.43
Iteracja 1100, najlepszy koszt: 14373, temperatura: 0.16
Iteracja 1200, najlepszy koszt: 14144, temperatura: 0.06
Iteracja 1300, najlepszy koszt: 13872, temperatura: 0.02
Iteracja 1400, najlepszy koszt: 13348, temperatura: 0.01
Iteracja 1500, najlepszy koszt: 12790, temperatura: 0.00
Iteracja 1600, najlepszy koszt: 12790, temperatura: 0.00
Iteracja 1700, najlepszy kos

In [93]:
opt_p_2, opt_c_2= simulated_annealing(
        data2,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )

Iteracja 0, najlepszy koszt: 542274.7237886683, temperatura: 9900.00
Iteracja 100, najlepszy koszt: 463021.17048022134, temperatura: 3623.72
Iteracja 200, najlepszy koszt: 372920.4720996408, temperatura: 1326.40
Iteracja 300, najlepszy koszt: 345936.905799256, temperatura: 485.50
Iteracja 400, najlepszy koszt: 305836.50563463825, temperatura: 177.71
Iteracja 500, najlepszy koszt: 291632.449347409, temperatura: 65.05
Iteracja 600, najlepszy koszt: 272003.5543813665, temperatura: 23.81
Iteracja 700, najlepszy koszt: 262177.59519099473, temperatura: 8.72
Iteracja 800, najlepszy koszt: 242476.23852048683, temperatura: 3.19
Iteracja 900, najlepszy koszt: 235190.8103608669, temperatura: 1.17
Iteracja 1000, najlepszy koszt: 227793.39684139556, temperatura: 0.43
Iteracja 1100, najlepszy koszt: 216073.4990141137, temperatura: 0.16
Iteracja 1200, najlepszy koszt: 212744.77760914166, temperatura: 0.06
Iteracja 1300, najlepszy koszt: 204965.1486123915, temperatura: 0.02
Iteracja 1400, najlepszy ko

In [94]:
opt_p_3, opt_c_3= simulated_annealing(
        data3,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )

Iteracja 0, najlepszy koszt: 669575.57897564, temperatura: 9900.00
Iteracja 100, najlepszy koszt: 590038.5593642683, temperatura: 3623.72
Iteracja 200, najlepszy koszt: 543553.894313313, temperatura: 1326.40
Iteracja 300, najlepszy koszt: 508215.3816900075, temperatura: 485.50
Iteracja 400, najlepszy koszt: 477698.6121935424, temperatura: 177.71
Iteracja 500, najlepszy koszt: 429765.64482637896, temperatura: 65.05
Iteracja 600, najlepszy koszt: 401945.02102190605, temperatura: 23.81
Iteracja 700, najlepszy koszt: 386028.8447689344, temperatura: 8.72
Iteracja 800, najlepszy koszt: 371195.6992623328, temperatura: 3.19
Iteracja 900, najlepszy koszt: 361612.6036991779, temperatura: 1.17
Iteracja 1000, najlepszy koszt: 357057.8789856272, temperatura: 0.43
Iteracja 1100, najlepszy koszt: 329163.32503587066, temperatura: 0.16
Iteracja 1200, najlepszy koszt: 317063.56118203205, temperatura: 0.06
Iteracja 1300, najlepszy koszt: 303416.1701706177, temperatura: 0.02
Iteracja 1400, najlepszy koszt

In [95]:
ins_p_1, ins_c_1= simulated_annealing(
        data1,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "insertion"
    )

Iteracja 0, najlepszy koszt: 51668, temperatura: 9900.00
Iteracja 100, najlepszy koszt: 44410, temperatura: 3623.72
Iteracja 200, najlepszy koszt: 44410, temperatura: 1326.40
Iteracja 300, najlepszy koszt: 36042, temperatura: 485.50
Iteracja 400, najlepszy koszt: 30438, temperatura: 177.71
Iteracja 500, najlepszy koszt: 26840, temperatura: 65.05
Iteracja 600, najlepszy koszt: 25713, temperatura: 23.81
Iteracja 700, najlepszy koszt: 23033, temperatura: 8.72
Iteracja 800, najlepszy koszt: 23025, temperatura: 3.19
Iteracja 900, najlepszy koszt: 21712, temperatura: 1.17
Iteracja 1000, najlepszy koszt: 21252, temperatura: 0.43
Iteracja 1100, najlepszy koszt: 20455, temperatura: 0.16
Iteracja 1200, najlepszy koszt: 18559, temperatura: 0.06
Iteracja 1300, najlepszy koszt: 17763, temperatura: 0.02
Iteracja 1400, najlepszy koszt: 17609, temperatura: 0.01
Iteracja 1500, najlepszy koszt: 17476, temperatura: 0.00
Iteracja 1600, najlepszy koszt: 17252, temperatura: 0.00
Iteracja 1700, najlepszy kos

In [96]:
ins_p_2, ins_c_2= simulated_annealing(
        data2,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "insertion"
    )

Iteracja 0, najlepszy koszt: 625731.1145281976, temperatura: 9900.00
Iteracja 100, najlepszy koszt: 484039.01289040013, temperatura: 3623.72
Iteracja 200, najlepszy koszt: 433988.7071417964, temperatura: 1326.40
Iteracja 300, najlepszy koszt: 357645.70085527835, temperatura: 485.50
Iteracja 400, najlepszy koszt: 318550.36371412774, temperatura: 177.71
Iteracja 500, najlepszy koszt: 302063.197057154, temperatura: 65.05
Iteracja 600, najlepszy koszt: 298358.99503885984, temperatura: 23.81
Iteracja 700, najlepszy koszt: 290915.0977926015, temperatura: 8.72
Iteracja 800, najlepszy koszt: 271699.36766204145, temperatura: 3.19
Iteracja 900, najlepszy koszt: 258536.67705274237, temperatura: 1.17
Iteracja 1000, najlepszy koszt: 255480.59561852526, temperatura: 0.43
Iteracja 1100, najlepszy koszt: 248940.93856342183, temperatura: 0.16
Iteracja 1200, najlepszy koszt: 246352.44317694308, temperatura: 0.06
Iteracja 1300, najlepszy koszt: 244572.24760765277, temperatura: 0.02
Iteracja 1400, najleps

In [97]:
ins_p_3, ins_c_3= simulated_annealing(
        data3,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "insertion"
    )

Iteracja 0, najlepszy koszt: 619298.3358314373, temperatura: 9900.00
Iteracja 100, najlepszy koszt: 587925.5633771485, temperatura: 3623.72
Iteracja 200, najlepszy koszt: 547309.0270562758, temperatura: 1326.40
Iteracja 300, najlepszy koszt: 501184.4466351283, temperatura: 485.50
Iteracja 400, najlepszy koszt: 458371.056716303, temperatura: 177.71
Iteracja 500, najlepszy koszt: 444664.67538996204, temperatura: 65.05
Iteracja 600, najlepszy koszt: 416334.80689344846, temperatura: 23.81
Iteracja 700, najlepszy koszt: 406018.3826743758, temperatura: 8.72
Iteracja 800, najlepszy koszt: 391367.6562500267, temperatura: 3.19
Iteracja 900, najlepszy koszt: 385255.20450679125, temperatura: 1.17
Iteracja 1000, najlepszy koszt: 374269.13588582654, temperatura: 0.43
Iteracja 1100, najlepszy koszt: 366483.78849623824, temperatura: 0.16
Iteracja 1200, najlepszy koszt: 364859.30367430235, temperatura: 0.06
Iteracja 1300, najlepszy koszt: 355593.5571875711, temperatura: 0.02
Iteracja 1400, najlepszy k

In [98]:
W10 = {
    "Swap" : [swap_c_1, swap_c_2, swap_c_3],
    "Opt": [opt_c_1, opt_c_2, opt_c_3],
    "Insertion" : [ins_c_1, ins_c_2, ins_c_3]
}

metody_SA = pd.DataFrame(W10)

Badanie wpływu parametru initial temperature

In [99]:
# DANE NR 1 
results = []
for initial_temperature in range(1000, 100000,  2000):
    _, best_cost =simulated_annealing(
        data1,
        initial_temperature= initial_temperature,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )
    results.append({"PARAMETR": initial_temperature, "WYNIK": best_cost})


tem1 = pd.DataFrame(results)


# DANE NR 2

results = []
for initial_temperature in range(1000, 100000,  2000):
    _, best_cost =simulated_annealing(
        data2,
        initial_temperature= initial_temperature,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )
    results.append({"PARAMETR": initial_temperature, "WYNIK": best_cost})

# Tworzenie ramki danych
tem2 = pd.DataFrame(results)


# DANE NR 3

results = []
for initial_temperature in range(1000, 100000,  2000):
    _, best_cost =simulated_annealing(
        data3,
        initial_temperature= initial_temperature,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )
    results.append({"PARAMETR": initial_temperature, "WYNIK": best_cost})

# Tworzenie ramki danych
tem3 = pd.DataFrame(results)

Iteracja 0, najlepszy koszt: 49394, temperatura: 990.00
Iteracja 100, najlepszy koszt: 39814, temperatura: 362.37
Iteracja 200, najlepszy koszt: 31208, temperatura: 132.64
Iteracja 300, najlepszy koszt: 27259, temperatura: 48.55
Iteracja 400, najlepszy koszt: 23289, temperatura: 17.77
Iteracja 500, najlepszy koszt: 19846, temperatura: 6.50
Iteracja 600, najlepszy koszt: 18477, temperatura: 2.38
Iteracja 700, najlepszy koszt: 17189, temperatura: 0.87
Iteracja 800, najlepszy koszt: 16713, temperatura: 0.32
Iteracja 900, najlepszy koszt: 15530, temperatura: 0.12
Iteracja 1000, najlepszy koszt: 15363, temperatura: 0.04
Iteracja 1100, najlepszy koszt: 15033, temperatura: 0.02
Iteracja 1200, najlepszy koszt: 14592, temperatura: 0.01
Iteracja 1300, najlepszy koszt: 14224, temperatura: 0.00
Iteracja 1400, najlepszy koszt: 14219, temperatura: 0.00
Iteracja 1500, najlepszy koszt: 13833, temperatura: 0.00
Iteracja 1600, najlepszy koszt: 13833, temperatura: 0.00
Iteracja 1700, najlepszy koszt: 137

In [100]:
merged_tem = tem1.merge(tem2, on="PARAMETR").merge(tem3, on="PARAMETR")

Badanie wpływu parametru cooling rate

In [102]:


# DANE NR 1
results = []
for cooling_rate in np.arange(0.8, 0.999, 0.01):
    _, best_cost = simulated_annealing(
        data1,
        initial_temperature=10000,
        cooling_rate=cooling_rate,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit=None,
        move_type="two_opt"
    )
    results.append({"PARAMETR": cooling_rate, "WYNIK": best_cost})

cool1 = pd.DataFrame(results)

# DANE NR 2
results = []
for cooling_rate in np.arange(0.8, 0.999, 0.01):
    _, best_cost = simulated_annealing(
        data2,
        initial_temperature=10000,
        cooling_rate=cooling_rate,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit=None,
        move_type="two_opt"
    )
    results.append({"PARAMETR": cooling_rate, "WYNIK": best_cost})

cool2 = pd.DataFrame(results)

# DANE NR 3
results = []
for cooling_rate in np.arange(0.8, 0.999, 0.01):
    _, best_cost = simulated_annealing(
        data3,
        initial_temperature=10000,
        cooling_rate=cooling_rate,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit=None,
        move_type="two_opt"
    )
    results.append({"PARAMETR": cooling_rate, "WYNIK": best_cost})

cool3 = pd.DataFrame(results)


Iteracja 0, najlepszy koszt: 53905, temperatura: 8000.00
Temperatura osiągnęła wartość minimalną.
Iteracja 0, najlepszy koszt: 54131, temperatura: 8100.00
Temperatura osiągnęła wartość minimalną.
Iteracja 0, najlepszy koszt: 53890, temperatura: 8200.00
Iteracja 100, najlepszy koszt: 31746, temperatura: 0.00
Temperatura osiągnęła wartość minimalną.
Iteracja 0, najlepszy koszt: 49763, temperatura: 8300.00
Iteracja 100, najlepszy koszt: 31684, temperatura: 0.00
Temperatura osiągnęła wartość minimalną.
Iteracja 0, najlepszy koszt: 49961, temperatura: 8400.00
Iteracja 100, najlepszy koszt: 28806, temperatura: 0.00
Temperatura osiągnęła wartość minimalną.
Iteracja 0, najlepszy koszt: 45963, temperatura: 8500.00
Iteracja 100, najlepszy koszt: 35718, temperatura: 0.00
Temperatura osiągnęła wartość minimalną.
Iteracja 0, najlepszy koszt: 53599, temperatura: 8600.00
Iteracja 100, najlepszy koszt: 35011, temperatura: 0.00
Temperatura osiągnęła wartość minimalną.
Iteracja 0, najlepszy koszt: 50210

In [103]:
merged_cool = cool1.merge(cool2, on="PARAMETR").merge(cool3, on="PARAMETR")

Badanie wpływu parametru max iteration

In [104]:
# DANE NR 1 
results = []
for max_iterations in range(100, 100000,  200):
    _, best_cost =simulated_annealing(
        data1,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=max_iterations,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost})


max1 = pd.DataFrame(results)


# DANE NR 2

results = []
for max_iterations in range(100, 100000,  200):
    _, best_cost =simulated_annealing(
        data2,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=max_iterations,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost})

# Tworzenie ramki danych
max2 = pd.DataFrame(results)


# DANE NR 3

results = []
for max_iterations in range(100, 100000,  200):
    _, best_cost =simulated_annealing(
        data3,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=max_iterations,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost})

# Tworzenie ramki danych
max3 = pd.DataFrame(results)

Iteracja 0, najlepszy koszt: 49929, temperatura: 9900.00
Iteracja 0, najlepszy koszt: 47323, temperatura: 9900.00
Iteracja 100, najlepszy koszt: 41355, temperatura: 3623.72
Iteracja 200, najlepszy koszt: 39338, temperatura: 1326.40
Iteracja 0, najlepszy koszt: 55886, temperatura: 9900.00
Iteracja 100, najlepszy koszt: 44411, temperatura: 3623.72
Iteracja 200, najlepszy koszt: 41877, temperatura: 1326.40
Iteracja 300, najlepszy koszt: 32211, temperatura: 485.50
Iteracja 400, najlepszy koszt: 32211, temperatura: 177.71
Iteracja 0, najlepszy koszt: 50924, temperatura: 9900.00
Iteracja 100, najlepszy koszt: 35637, temperatura: 3623.72
Iteracja 200, najlepszy koszt: 35637, temperatura: 1326.40
Iteracja 300, najlepszy koszt: 35637, temperatura: 485.50
Iteracja 400, najlepszy koszt: 30420, temperatura: 177.71
Iteracja 500, najlepszy koszt: 25775, temperatura: 65.05
Iteracja 600, najlepszy koszt: 22825, temperatura: 23.81
Iteracja 0, najlepszy koszt: 52368, temperatura: 9900.00
Iteracja 100, n

In [105]:
merged_max = max1.merge(max2, on="PARAMETR").merge(max3, on="PARAMETR")

Badanie wpływu parametru min temperature

In [106]:
# DANE NR 1
results = []
for min_temperature in np.logspace(-6, -2, num=10):  
    _, best_cost = simulated_annealing(
        data1,
        initial_temperature=10000,
        cooling_rate=0.99,
        max_iterations=10000, 
        min_temperature=min_temperature,
        time_limit=None,
        move_type="two_opt"
    )
    results.append({"PARAMETR": min_temperature, "WYNIK": best_cost})

min_temp1 = pd.DataFrame(results)

# DANE NR 2
results = []
for min_temperature in np.logspace(-6, -2, num=10):
    _, best_cost = simulated_annealing(
        data2,
        initial_temperature=10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=min_temperature,
        time_limit=None,
        move_type="two_opt"
    )
    results.append({"PARAMETR": min_temperature, "WYNIK": best_cost})

min_temp2 = pd.DataFrame(results)

# DANE NR 3
results = []
for min_temperature in np.logspace(-6, -2, num=10):
    _, best_cost = simulated_annealing(
        data3,
        initial_temperature=10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=min_temperature,
        time_limit=None,
        move_type="two_opt"
    )
    results.append({"PARAMETR": min_temperature, "WYNIK": best_cost})

min_temp3 = pd.DataFrame(results)

Iteracja 0, najlepszy koszt: 48927, temperatura: 9900.00
Iteracja 100, najlepszy koszt: 44057, temperatura: 3623.72
Iteracja 200, najlepszy koszt: 44057, temperatura: 1326.40
Iteracja 300, najlepszy koszt: 40245, temperatura: 485.50
Iteracja 400, najlepszy koszt: 32437, temperatura: 177.71
Iteracja 500, najlepszy koszt: 24906, temperatura: 65.05
Iteracja 600, najlepszy koszt: 21209, temperatura: 23.81
Iteracja 700, najlepszy koszt: 19530, temperatura: 8.72
Iteracja 800, najlepszy koszt: 17935, temperatura: 3.19
Iteracja 900, najlepszy koszt: 16205, temperatura: 1.17
Iteracja 1000, najlepszy koszt: 14847, temperatura: 0.43
Iteracja 1100, najlepszy koszt: 14833, temperatura: 0.16
Iteracja 1200, najlepszy koszt: 14549, temperatura: 0.06
Iteracja 1300, najlepszy koszt: 13970, temperatura: 0.02
Iteracja 1400, najlepszy koszt: 13722, temperatura: 0.01
Iteracja 1500, najlepszy koszt: 13175, temperatura: 0.00
Iteracja 1600, najlepszy koszt: 13160, temperatura: 0.00
Iteracja 1700, najlepszy kos

In [107]:
merged_min = min_temp1.merge(min_temp2, on="PARAMETR").merge(min_temp3, on="PARAMETR")

In [115]:
resul = {
    "Porównanie_metod": metody_SA,
    "max_iterations": merged_max,
    "min_tem": merged_min,
    "cooling_rate": merged_cool,
    "initial_temp": merged_tem
}

# Ścieżka do pliku Excel
file_name = "SA.xlsx"

# Zapisujemy dane do Excela
with pd.ExcelWriter(file_name) as writer:
    for sheet_name, df in resul.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Wyniki zostały zapisane w pliku {file_name}")

Wyniki zostały zapisane w pliku SA.xlsx


## Algorytm przeszukiwania Tabu (TS)

In [5]:
def tabu_search(distance_matrix, tabu_list_length=10, max_iterations=50, max_no_improve=1000, move_type="two_opt", time_limit=1000):
    n = len(distance_matrix)

    # initial solution
    best_solution = list(range(n))
    random.shuffle(best_solution)
    best_cost = calculate_path_cost(distance_matrix, best_solution)
    current_solution = best_solution[:]
    current_cost = best_cost
 
    tabu_list = deque(maxlen=tabu_list_length)
    start_time = time.time()
    no_improve = 0
 
    move_generators = {
        "swap": swap_move,
        "two_opt": two_opt_move,
        "insertion": insertion_move
    }
 
    if move_type not in move_generators:
        raise ValueError(f"Nieznany typ ruchu: {move_type}")
 
    move_generator = move_generators[move_type]
 
    for iteration in range(max_iterations):
        if time_limit and (time.time() - start_time > time_limit):
            print("Przekroczono limit czasu.")
            break

        neighborhood = list(move_generator(current_solution))
        neighborhood_costs = [calculate_path_cost(distance_matrix, neighbor) for neighbor in neighborhood]
 
        best_neighbor = None
        best_neighbor_cost = float('inf')
 
        for neighbor, cost in zip(neighborhood, neighborhood_costs):
            if (neighbor not in tabu_list) and (cost < best_neighbor_cost):
                best_neighbor = neighbor
                best_neighbor_cost = cost
 
        if best_neighbor is None:
            continue
 
        current_solution = best_neighbor
        current_cost = best_neighbor_cost
        tabu_list.append(current_solution)
 
        if current_cost < best_cost:
            best_solution = current_solution[:]
            best_cost = current_cost
            no_improve = 0
        else:
            no_improve += 1

        if no_improve >= max_no_improve:
            print("Osiągnięto limit iteracji bez poprawy.")
            break    

        if iteration % 10 == 0:
            elapsed_time = time.time() - start_time
            print(f"Iteration {iteration}, Best Cost: {best_cost:.2f}, Elapsed Time: {elapsed_time:.2f} seconds")    
 
    return best_solution, best_cost
 

### Generowanie wyników

Za podstawowe dane przyjmujemy :

* tabu_list_length = 10,

* max_iterations = 10000,

* max_no_improve = 50,

* move_type = two_opt,

* time_limit = 1000

Nastepnie badane są wpływy poszczególnych parametrów 

Badanie wpływu zmian wartości parametru *move_type*

In [8]:
swap_p_1, swap_c_1 = tabu_search(
    data1,
    tabu_list_length=5,
    max_iterations=1000,
    max_no_improve=50,
    move_type="swap",
    time_limit=1000
)


Iteration 0, Best Cost: 43648.00, Elapsed Time: 0.01 seconds
Iteration 10, Best Cost: 25149.00, Elapsed Time: 0.13 seconds
Iteration 20, Best Cost: 20046.00, Elapsed Time: 0.27 seconds
Iteration 30, Best Cost: 17510.00, Elapsed Time: 0.43 seconds
Iteration 40, Best Cost: 17081.00, Elapsed Time: 0.65 seconds
Iteration 50, Best Cost: 16485.00, Elapsed Time: 0.97 seconds
Iteration 60, Best Cost: 16229.00, Elapsed Time: 1.17 seconds
Iteration 70, Best Cost: 16229.00, Elapsed Time: 1.35 seconds
Iteration 80, Best Cost: 16229.00, Elapsed Time: 1.53 seconds
Iteration 90, Best Cost: 16229.00, Elapsed Time: 1.75 seconds
Iteration 100, Best Cost: 16229.00, Elapsed Time: 2.09 seconds
Osiągnięto limit iteracji bez poprawy.


In [7]:
swap_p_2, swap_c_2 = tabu_search(
    data2,
    tabu_list_length=10,
    max_iterations=10000,
    max_no_improve=50,
    move_type="swap",
    time_limit=1000
)


Iteration 0, Best Cost: 582237.98, Elapsed Time: 0.08 seconds
Iteration 10, Best Cost: 360765.77, Elapsed Time: 0.70 seconds
Iteration 20, Best Cost: 270931.29, Elapsed Time: 1.35 seconds
Iteration 30, Best Cost: 236904.47, Elapsed Time: 2.09 seconds
Iteration 40, Best Cost: 216468.96, Elapsed Time: 3.15 seconds
Iteration 50, Best Cost: 202086.56, Elapsed Time: 4.13 seconds
Iteration 60, Best Cost: 184085.97, Elapsed Time: 4.86 seconds
Iteration 70, Best Cost: 175437.35, Elapsed Time: 5.53 seconds
Iteration 80, Best Cost: 170571.10, Elapsed Time: 6.20 seconds
Iteration 90, Best Cost: 165185.93, Elapsed Time: 6.86 seconds
Iteration 100, Best Cost: 165129.52, Elapsed Time: 7.55 seconds
Iteration 110, Best Cost: 165129.52, Elapsed Time: 8.24 seconds
Iteration 120, Best Cost: 165129.52, Elapsed Time: 8.87 seconds
Iteration 130, Best Cost: 165129.52, Elapsed Time: 9.64 seconds
Iteration 140, Best Cost: 165129.52, Elapsed Time: 10.31 seconds
Osiągnięto limit iteracji bez poprawy.


In [6]:
swap_p_3, swap_c_3 = tabu_search(
    data3,
    tabu_list_length=10,
    max_iterations=10000,
    max_no_improve=50,
    move_type="swap",
    time_limit=1000
)


Iteration 0, Best Cost: 596291.67, Elapsed Time: 0.25 seconds
Iteration 10, Best Cost: 441542.08, Elapsed Time: 2.83 seconds
Iteration 20, Best Cost: 357328.31, Elapsed Time: 5.66 seconds
Iteration 30, Best Cost: 309230.08, Elapsed Time: 8.31 seconds
Iteration 40, Best Cost: 276088.26, Elapsed Time: 10.93 seconds
Iteration 50, Best Cost: 258236.73, Elapsed Time: 13.66 seconds
Iteration 60, Best Cost: 247231.29, Elapsed Time: 16.15 seconds
Iteration 70, Best Cost: 232045.37, Elapsed Time: 18.91 seconds
Iteration 80, Best Cost: 223226.24, Elapsed Time: 21.99 seconds
Iteration 90, Best Cost: 218245.62, Elapsed Time: 24.65 seconds
Iteration 100, Best Cost: 214067.72, Elapsed Time: 27.24 seconds
Iteration 110, Best Cost: 210356.52, Elapsed Time: 29.84 seconds
Iteration 120, Best Cost: 209417.77, Elapsed Time: 32.30 seconds
Iteration 130, Best Cost: 208274.22, Elapsed Time: 34.79 seconds
Iteration 140, Best Cost: 205718.44, Elapsed Time: 37.83 seconds
Iteration 150, Best Cost: 204173.52, Ela

In [9]:
two_opt_p_1, two_opt_c_1 = tabu_search(
    data1,
    tabu_list_length=10,
    max_iterations=10000,
    max_no_improve=50,
    move_type="two_opt",
    time_limit=1000
)


Iteration 0, Best Cost: 44346.00, Elapsed Time: 0.02 seconds
Iteration 10, Best Cost: 22866.00, Elapsed Time: 0.15 seconds
Iteration 20, Best Cost: 14512.00, Elapsed Time: 0.29 seconds
Iteration 30, Best Cost: 11877.00, Elapsed Time: 0.48 seconds
Iteration 40, Best Cost: 11278.00, Elapsed Time: 0.65 seconds
Iteration 50, Best Cost: 11102.00, Elapsed Time: 0.82 seconds
Iteration 60, Best Cost: 11102.00, Elapsed Time: 0.99 seconds
Iteration 70, Best Cost: 11102.00, Elapsed Time: 1.16 seconds
Iteration 80, Best Cost: 11102.00, Elapsed Time: 1.31 seconds
Iteration 90, Best Cost: 11102.00, Elapsed Time: 1.46 seconds
Osiągnięto limit iteracji bez poprawy.


In [10]:
two_opt_p_2, two_opt_c_2 = tabu_search(
    data2,
    tabu_list_length=10,
    max_iterations=10000,
    max_no_improve=50,
    move_type="two_opt",
    time_limit=1000
)


Iteration 0, Best Cost: 569939.99, Elapsed Time: 0.06 seconds
Iteration 10, Best Cost: 385190.13, Elapsed Time: 0.76 seconds
Iteration 20, Best Cost: 270757.12, Elapsed Time: 1.48 seconds
Iteration 30, Best Cost: 202672.73, Elapsed Time: 2.12 seconds
Iteration 40, Best Cost: 161493.63, Elapsed Time: 2.74 seconds
Iteration 50, Best Cost: 134837.63, Elapsed Time: 3.36 seconds
Iteration 60, Best Cost: 121409.81, Elapsed Time: 3.91 seconds
Iteration 70, Best Cost: 111333.67, Elapsed Time: 4.51 seconds
Iteration 80, Best Cost: 110358.92, Elapsed Time: 5.06 seconds
Iteration 90, Best Cost: 110358.92, Elapsed Time: 5.61 seconds
Iteration 100, Best Cost: 110358.92, Elapsed Time: 6.15 seconds
Iteration 110, Best Cost: 110358.92, Elapsed Time: 6.73 seconds
Iteration 120, Best Cost: 110358.92, Elapsed Time: 7.39 seconds
Osiągnięto limit iteracji bez poprawy.


In [11]:
two_opt_p_3, two_opt_c_3 = tabu_search(
    data3,
    tabu_list_length=10,
    max_iterations=10000,
    max_no_improve=50,
    move_type="two_opt",
    time_limit=1000
)

Iteration 0, Best Cost: 608343.74, Elapsed Time: 0.21 seconds
Iteration 10, Best Cost: 465125.80, Elapsed Time: 2.87 seconds
Iteration 20, Best Cost: 372618.92, Elapsed Time: 5.62 seconds
Iteration 30, Best Cost: 300265.49, Elapsed Time: 8.13 seconds
Iteration 40, Best Cost: 246700.79, Elapsed Time: 10.77 seconds
Iteration 50, Best Cost: 210474.95, Elapsed Time: 13.74 seconds
Iteration 60, Best Cost: 184885.14, Elapsed Time: 16.66 seconds
Iteration 70, Best Cost: 163104.78, Elapsed Time: 19.20 seconds
Iteration 80, Best Cost: 146797.60, Elapsed Time: 21.68 seconds
Iteration 90, Best Cost: 137396.31, Elapsed Time: 24.27 seconds
Iteration 100, Best Cost: 132109.52, Elapsed Time: 26.88 seconds
Iteration 110, Best Cost: 129339.90, Elapsed Time: 29.42 seconds
Iteration 120, Best Cost: 127989.14, Elapsed Time: 32.00 seconds
Iteration 130, Best Cost: 127283.02, Elapsed Time: 34.62 seconds
Iteration 140, Best Cost: 126553.83, Elapsed Time: 37.45 seconds
Iteration 150, Best Cost: 126388.67, Ela

In [12]:
insertion_p_1, insertion_c_1 = tabu_search(
    data1,
    tabu_list_length=10,
    max_iterations=10000,
    max_no_improve=50,
    move_type="insertion",
    time_limit=1000
)


Iteration 0, Best Cost: 47672.00, Elapsed Time: 0.04 seconds
Iteration 10, Best Cost: 26514.00, Elapsed Time: 0.43 seconds
Iteration 20, Best Cost: 20178.00, Elapsed Time: 0.83 seconds
Iteration 30, Best Cost: 17155.00, Elapsed Time: 1.19 seconds
Iteration 40, Best Cost: 14667.00, Elapsed Time: 1.51 seconds
Iteration 50, Best Cost: 14010.00, Elapsed Time: 1.83 seconds
Iteration 60, Best Cost: 13486.00, Elapsed Time: 2.21 seconds
Iteration 70, Best Cost: 13047.00, Elapsed Time: 2.54 seconds
Iteration 80, Best Cost: 12960.00, Elapsed Time: 2.90 seconds
Iteration 90, Best Cost: 12953.00, Elapsed Time: 3.28 seconds
Iteration 100, Best Cost: 12923.00, Elapsed Time: 3.62 seconds
Iteration 110, Best Cost: 12923.00, Elapsed Time: 4.01 seconds
Iteration 120, Best Cost: 12923.00, Elapsed Time: 4.33 seconds
Iteration 130, Best Cost: 12923.00, Elapsed Time: 4.64 seconds
Iteration 140, Best Cost: 12923.00, Elapsed Time: 4.91 seconds
Osiągnięto limit iteracji bez poprawy.


In [13]:
insertion_p_2, insertion_c_2 = tabu_search(
    data2,
    tabu_list_length=10,
    max_iterations=10000,
    max_no_improve=50,
    move_type="insertion",
    time_limit=1000
)


Iteration 0, Best Cost: 535099.76, Elapsed Time: 0.11 seconds
Iteration 10, Best Cost: 355379.93, Elapsed Time: 1.27 seconds
Iteration 20, Best Cost: 263286.61, Elapsed Time: 2.45 seconds
Iteration 30, Best Cost: 216824.95, Elapsed Time: 3.70 seconds
Iteration 40, Best Cost: 180439.22, Elapsed Time: 4.89 seconds
Iteration 50, Best Cost: 163176.98, Elapsed Time: 6.05 seconds
Iteration 60, Best Cost: 144931.64, Elapsed Time: 7.25 seconds
Iteration 70, Best Cost: 140513.49, Elapsed Time: 8.60 seconds
Iteration 80, Best Cost: 139951.16, Elapsed Time: 10.05 seconds
Iteration 90, Best Cost: 134569.91, Elapsed Time: 11.32 seconds
Iteration 100, Best Cost: 129916.19, Elapsed Time: 12.73 seconds
Iteration 110, Best Cost: 127034.52, Elapsed Time: 14.05 seconds
Iteration 120, Best Cost: 127034.52, Elapsed Time: 15.40 seconds
Iteration 130, Best Cost: 126867.68, Elapsed Time: 16.74 seconds
Iteration 140, Best Cost: 125985.68, Elapsed Time: 18.20 seconds
Iteration 150, Best Cost: 125985.68, Elapsed

In [14]:
insertion_p_3, insertion_c_3 = tabu_search(
    data3,
    tabu_list_length=10,
    max_iterations=10000,
    max_no_improve=50,
    move_type="insertion",
    time_limit=1000
)

Iteration 0, Best Cost: 637952.95, Elapsed Time: 0.48 seconds
Iteration 10, Best Cost: 507494.51, Elapsed Time: 5.55 seconds
Iteration 20, Best Cost: 420816.40, Elapsed Time: 11.08 seconds
Iteration 30, Best Cost: 357175.86, Elapsed Time: 16.88 seconds
Iteration 40, Best Cost: 311096.43, Elapsed Time: 22.61 seconds
Iteration 50, Best Cost: 276647.99, Elapsed Time: 28.22 seconds
Iteration 60, Best Cost: 250664.68, Elapsed Time: 33.95 seconds
Iteration 70, Best Cost: 232632.15, Elapsed Time: 39.39 seconds
Iteration 80, Best Cost: 212861.54, Elapsed Time: 45.25 seconds
Iteration 90, Best Cost: 200240.52, Elapsed Time: 51.37 seconds
Iteration 100, Best Cost: 191352.72, Elapsed Time: 56.72 seconds
Iteration 110, Best Cost: 176102.26, Elapsed Time: 61.69 seconds
Iteration 120, Best Cost: 167590.36, Elapsed Time: 66.85 seconds
Iteration 130, Best Cost: 161092.42, Elapsed Time: 72.12 seconds
Iteration 140, Best Cost: 154930.01, Elapsed Time: 78.16 seconds
Iteration 150, Best Cost: 150466.63, E

Tworzenie ramki danych dla TS

In [17]:
W11 = {
    "Swap" : [swap_c_1, swap_c_2, swap_c_3],
    "Opt": [two_opt_c_1, two_opt_c_2, two_opt_c_3],
    "Insertion" : [insertion_c_1, insertion_c_2, insertion_c_3]
}

metody_TS = pd.DataFrame(W11)

Badanie wpływu zmian wartości parametru *tabu_list_length*

In [18]:
tabu_tendures = [5, 10, 15, 20]

# DANE NR 1 
results = []
for tabu_list_length in tabu_tendures:
    _, best_cost = tabu_search(
        data1,
        max_iterations=10000,
        max_no_improve=50,
        move_type="two_opt",
        time_limit=1000,
        tabu_list_length=tabu_list_length,
    )
    results.append({"PARAMETR": tabu_list_length, "WYNIK": best_cost})


tabu1 = pd.DataFrame(results)


# DANE NR 2

results = []
for tabu_list_length in tabu_tendures:
    _, best_cost = tabu_search(
        data2,
        max_iterations=10000,
        max_no_improve=50,
        move_type="two_opt",
        time_limit=1000,
        tabu_list_length=tabu_list_length,
    )
    results.append({"PARAMETR": tabu_list_length, "WYNIK": best_cost})

# Tworzenie ramki danych
tabu2 = pd.DataFrame(results)


# DANE NR 3

results = []
for tabu_list_length in tabu_tendures:
    _, best_cost = tabu_search(
        data3,
        max_iterations=10000,
        max_no_improve=50,
        move_type="two_opt",
        time_limit=1000,
        tabu_list_length=tabu_list_length,
    )
    results.append({"PARAMETR": tabu_list_length, "WYNIK": best_cost})

# Tworzenie ramki danych
tabu3 = pd.DataFrame(results)

Iteration 0, Best Cost: 48077.00, Elapsed Time: 0.01 seconds
Iteration 10, Best Cost: 24587.00, Elapsed Time: 0.11 seconds
Iteration 20, Best Cost: 16817.00, Elapsed Time: 0.29 seconds
Iteration 30, Best Cost: 13056.00, Elapsed Time: 0.40 seconds
Iteration 40, Best Cost: 11716.00, Elapsed Time: 0.54 seconds
Iteration 50, Best Cost: 11610.00, Elapsed Time: 0.64 seconds
Iteration 60, Best Cost: 11610.00, Elapsed Time: 0.74 seconds
Iteration 70, Best Cost: 11610.00, Elapsed Time: 0.85 seconds
Iteration 80, Best Cost: 11610.00, Elapsed Time: 0.95 seconds
Iteration 90, Best Cost: 11610.00, Elapsed Time: 1.05 seconds
Osiągnięto limit iteracji bez poprawy.
Iteration 0, Best Cost: 48444.00, Elapsed Time: 0.01 seconds
Iteration 10, Best Cost: 27972.00, Elapsed Time: 0.13 seconds
Iteration 20, Best Cost: 17308.00, Elapsed Time: 0.25 seconds
Iteration 30, Best Cost: 12840.00, Elapsed Time: 0.35 seconds
Iteration 40, Best Cost: 11543.00, Elapsed Time: 0.48 seconds
Iteration 50, Best Cost: 11114.00

In [19]:
merged_tabu = tabu1.merge(tabu2, on="PARAMETR").merge(tabu3, on="PARAMETR")

Badanie wpływu zmian wartości parametru *max_iterations*

In [20]:
max_iterations_values = [500, 1000, 1500, 2500]

# DANE NR 1 
results = []
for max_iterations in max_iterations_values:
    _, best_cost = tabu_search(
        data1,
        max_iterations=max_iterations,
        max_no_improve=50,
        move_type="two_opt",
        time_limit=1000,
        tabu_list_length=10,
    )
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost})


max1 = pd.DataFrame(results)


# DANE NR 2

results = []
for max_iterations in max_iterations_values:
    _, best_cost = tabu_search(
        data2,
        max_iterations=max_iterations,
        max_no_improve=50,
        move_type="two_opt",
        time_limit=1000,
        tabu_list_length=10,
    )
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost})

# Tworzenie ramki danych
max2 = pd.DataFrame(results)


# DANE NR 3

results = []
for max_iterations in max_iterations_values:
    _, best_cost = tabu_search(
        data3,
        max_iterations=max_iterations,
        max_no_improve=50,
        move_type="two_opt",
        time_limit=1000,
        tabu_list_length=10,
    )
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost})

# Tworzenie ramki danych
max3 = pd.DataFrame(results)

Iteration 0, Best Cost: 46721.00, Elapsed Time: 0.01 seconds
Iteration 10, Best Cost: 22809.00, Elapsed Time: 0.19 seconds
Iteration 20, Best Cost: 14934.00, Elapsed Time: 0.44 seconds
Iteration 30, Best Cost: 12029.00, Elapsed Time: 0.62 seconds
Iteration 40, Best Cost: 11402.00, Elapsed Time: 0.77 seconds
Iteration 50, Best Cost: 11327.00, Elapsed Time: 0.89 seconds
Iteration 60, Best Cost: 11327.00, Elapsed Time: 1.08 seconds
Iteration 70, Best Cost: 11327.00, Elapsed Time: 1.33 seconds
Iteration 80, Best Cost: 11327.00, Elapsed Time: 1.51 seconds
Iteration 90, Best Cost: 11327.00, Elapsed Time: 1.68 seconds
Osiągnięto limit iteracji bez poprawy.
Iteration 0, Best Cost: 47484.00, Elapsed Time: 0.01 seconds
Iteration 10, Best Cost: 24352.00, Elapsed Time: 0.17 seconds
Iteration 20, Best Cost: 15833.00, Elapsed Time: 0.34 seconds
Iteration 30, Best Cost: 12583.00, Elapsed Time: 0.51 seconds
Iteration 40, Best Cost: 11525.00, Elapsed Time: 0.69 seconds
Iteration 50, Best Cost: 11376.00

In [21]:
merged_max = max1.merge(max2, on="PARAMETR").merge(max3, on="PARAMETR")

Badanie wpływu zmian wartości parametru *max_no_improve*

In [23]:
max_no_improve_values = [10, 50, 150, 250]

# DANE NR 1 
results = []
for max_no_improve in max_no_improve_values:
    _, best_cost = tabu_search(
        data1,
        max_iterations=10000,
        max_no_improve=max_no_improve,
        move_type="two_opt",
        time_limit=1000,
        tabu_list_length=10,
    )
    results.append({"PARAMETR": max_no_improve, "WYNIK": best_cost})


max_no_improve1 = pd.DataFrame(results)


# DANE NR 2

results = []
for max_no_improve in max_no_improve_values:
    _, best_cost = tabu_search(
        data2,
        max_iterations=10000,
        max_no_improve=max_no_improve,
        move_type="two_opt",
        time_limit=1000,
        tabu_list_length=10,
    )
    results.append({"PARAMETR": max_no_improve, "WYNIK": best_cost})

# Tworzenie ramki danych
max_no_improve2 = pd.DataFrame(results)


# DANE NR 3

results = []
for max_no_improve in max_no_improve_values:
    _, best_cost = tabu_search(
        data3,
        max_iterations=10000,
        max_no_improve=max_no_improve,
        move_type="two_opt",
        time_limit=1000,
        tabu_list_length=10,
    )
    results.append({"PARAMETR": max_no_improve, "WYNIK": best_cost})

# Tworzenie ramki danych
max_no_improve3 = pd.DataFrame(results)

Iteration 0, Best Cost: 46580.00, Elapsed Time: 0.02 seconds
Iteration 10, Best Cost: 22413.00, Elapsed Time: 0.19 seconds
Iteration 20, Best Cost: 14992.00, Elapsed Time: 0.37 seconds
Iteration 30, Best Cost: 11861.00, Elapsed Time: 0.54 seconds
Iteration 40, Best Cost: 10962.00, Elapsed Time: 0.70 seconds
Osiągnięto limit iteracji bez poprawy.
Iteration 0, Best Cost: 48055.00, Elapsed Time: 0.02 seconds
Iteration 10, Best Cost: 23637.00, Elapsed Time: 0.19 seconds
Iteration 20, Best Cost: 14968.00, Elapsed Time: 0.38 seconds
Iteration 30, Best Cost: 12259.00, Elapsed Time: 0.53 seconds
Iteration 40, Best Cost: 11931.00, Elapsed Time: 0.75 seconds
Iteration 50, Best Cost: 11602.00, Elapsed Time: 0.89 seconds
Iteration 60, Best Cost: 11271.00, Elapsed Time: 1.04 seconds
Iteration 70, Best Cost: 11271.00, Elapsed Time: 1.17 seconds
Iteration 80, Best Cost: 11271.00, Elapsed Time: 1.31 seconds
Iteration 90, Best Cost: 11271.00, Elapsed Time: 1.45 seconds
Iteration 100, Best Cost: 11271.0

In [24]:
merged_max_no_improve = max_no_improve1.merge(max_no_improve2, on="PARAMETR").merge(max_no_improve3, on="PARAMETR")

Badanie wpływu zmian wartości parametru *time_limit*

In [28]:
time_values = [60, 240, 500, 1000]

# DANE NR 1 
results = []
for time_limit in time_values:
    _, best_cost = tabu_search(
        data1,
        max_iterations=10000,
        max_no_improve=50,
        move_type="two_opt",
        time_limit=time_limit,
        tabu_list_length=10,
    )
    results.append({"PARAMETR": time_limit, "WYNIK": best_cost})


time1 = pd.DataFrame(results)


# DANE NR 2

results = []
for time_limit in time_values:
    _, best_cost = tabu_search(
        data2,
        max_iterations=10000,
        max_no_improve=50,
        move_type="two_opt",
        time_limit=time_limit,
        tabu_list_length=10,
    )
    results.append({"PARAMETR": time_limit, "WYNIK": best_cost})
# Tworzenie ramki danych
time2 = pd.DataFrame(results)


# DANE NR 3

results = []
for time_limit in time_values:
    _, best_cost = tabu_search(
        data3,
        max_iterations=10000,
        max_no_improve=50,
        move_type="two_opt",
        time_limit=time_limit,
        tabu_list_length=10,
    )
    results.append({"PARAMETR": time_limit, "WYNIK": best_cost})

# Tworzenie ramki danych
time3 = pd.DataFrame(results)

Iteration 0, Best Cost: 48423.00, Elapsed Time: 0.01 seconds
Iteration 10, Best Cost: 23685.00, Elapsed Time: 0.16 seconds
Iteration 20, Best Cost: 15764.00, Elapsed Time: 0.32 seconds
Iteration 30, Best Cost: 12835.00, Elapsed Time: 0.49 seconds
Iteration 40, Best Cost: 11751.00, Elapsed Time: 0.73 seconds
Iteration 50, Best Cost: 11751.00, Elapsed Time: 0.87 seconds
Iteration 60, Best Cost: 11751.00, Elapsed Time: 0.99 seconds
Iteration 70, Best Cost: 11751.00, Elapsed Time: 1.13 seconds
Iteration 80, Best Cost: 11751.00, Elapsed Time: 1.24 seconds
Osiągnięto limit iteracji bez poprawy.
Iteration 0, Best Cost: 47645.00, Elapsed Time: 0.01 seconds
Iteration 10, Best Cost: 23647.00, Elapsed Time: 0.13 seconds
Iteration 20, Best Cost: 15446.00, Elapsed Time: 0.27 seconds
Iteration 30, Best Cost: 12244.00, Elapsed Time: 0.38 seconds
Iteration 40, Best Cost: 11193.00, Elapsed Time: 0.50 seconds
Iteration 50, Best Cost: 11193.00, Elapsed Time: 0.63 seconds
Iteration 60, Best Cost: 11193.00

In [29]:
merged_time = time1.merge(time2, on="PARAMETR").merge(time3, on="PARAMETR")

Zapis wyników do pliku 

In [30]:
result = {
    "Porównanie_metod": metody_TS,
    "tabu_list_length": merged_tabu,
    "max_iterations": merged_max,
    "max_no_improve": merged_max_no_improve,
    "time_limit": merged_time 
}

# Ścieżka do pliku Excel
file_name = "TS.xlsx"

# Zapisujemy dane do Excela
with pd.ExcelWriter(file_name) as writer:
    for sheet_name, df in result.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Wyniki zostały zapisane w pliku {file_name}")

Wyniki zostały zapisane w pliku TS.xlsx


## Algorytm genetyczny

In [51]:
def crossover_order(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None] * size
    child[start:end] = parent1[start:end]

    pointer = 0
    for gene in parent2:
        if gene not in child:
            while child[pointer] is not None:
                pointer += 1
            child[pointer] = gene

    return child

def crossover_pmx(parent1, parent2):
    size = len(parent1)
    start, end = 2, 6
    child = [None] * size

    # Copy segment from parent1 to child
    child[start:end] = parent1[start:end]

    # Fill remaining positions from parent2
    for i in range(size):
        if child[i] is None:
            gene = parent2[i]
            while gene in child:
                gene = parent2[parent1.index(gene)]
            child[i] = gene

    return child

def selection_tournament(population, scores, k=3):
    selected = random.sample(range(len(population)), k)
    selected = sorted(selected, key=lambda x: scores[x])
    return population[selected[0]]

def selection_roulette(population, scores):
    total_score = sum(scores)
    
    # Calculate probabilities
    probabilities = [score / total_score for score in scores]
    print(probabilities)
    # Calculate cumulative probabilities
    cumulative_probabilities = [sum(probabilities[:i+1]) for i in range(len(probabilities))]
    print(cumulative_probabilities)
    # Pick a random number between 0 and 1
    pick = random.uniform(0, 1)
    print(pick)
    # Select the individual based on the random number
    for i, cumulative_probability in enumerate(cumulative_probabilities):
        if pick <= cumulative_probability:
            return population[i]


def mutate_swap(individual):
    i, j = random.sample(range(len(individual)), 2)
    individual[i], individual[j] = individual[j], individual[i]

def mutate_inversion(individual):
    i, j = sorted(random.sample(range(len(individual)), 2))
    individual[i:j] = reversed(individual[i:j])

def mutate_insertion(individual):
    i, j = random.sample(range(len(individual)), 2)
    gene = individual.pop(i)
    individual.insert(j, gene)


def genetic_algorithm(distance_matrix, population_size=25, generations=500, crossover_rate=0.7, mutation_rate=0.2, selection_method='tournament', crossover_method='order', mutation_method='swap'):
    start_time = time.time()

    n = len(distance_matrix)
    population = [random.sample(range(n), n) for _ in range(population_size)]
    best_solution = None
    best_cost = float('inf')

    for generation in range(generations):
        scores = [calculate_path_cost(distance_matrix, individual) for individual in population]
        new_population = []

        for _ in range(population_size // 2):
            if selection_method == 'tournament':
                parent1 = selection_tournament(population, scores)
                parent2 = selection_tournament(population, scores)
            elif selection_method == 'roulette':
                parent1 = selection_roulette(population, scores)
                parent2 = selection_roulette(population, scores)

            if random.random() < crossover_rate:
                if crossover_method == 'order':
                    child1 = crossover_order(parent1, parent2)
                    child2 = crossover_order(parent2, parent1)
                elif crossover_method == 'pmx':
                    child1 = crossover_pmx(parent1, parent2)
                    child2 = crossover_pmx(parent2, parent1)
            else:
                child1, child2 = parent1[:], parent2[:]

            if random.random() < mutation_rate:
                if mutation_method == 'swap':
                    mutate_swap(child1)
                    mutate_swap(child2)
                elif mutation_method == 'inversion':
                    mutate_inversion(child1)
                    mutate_inversion(child2)
                elif mutation_method == 'insertion':
                    mutate_insertion(child1)
                    mutate_insertion(child2)

            new_population.extend([child1, child2])

        population = new_population

        for individual in population:
            cost = calculate_path_cost(distance_matrix, individual)
            if cost < best_cost:
                best_time = time.time() - start_time
                best_solution = individual
                best_cost = cost

        if generation % 100 == 0:
            elapsed_time = time.time() - start_time  # Calculate elapsed time
            print(f"Iteration {generation}, Best Cost: {best_cost}, Elapsed Time: {elapsed_time:.2f} seconds")

    return best_solution, best_cost, best_time

# Example usage
best_path, best_cost, best_time = genetic_algorithm(
    data1,
    population_size=25,
    generations=500,
    crossover_rate=0.8,
    mutation_rate=0.2,
    selection_method='tournament',
    crossover_method='order',
    mutation_method='swap'
)

Generation 0, Best Cost: 45693
Generation 100, Best Cost: 24092
Generation 200, Best Cost: 20104
Generation 300, Best Cost: 18654
Generation 400, Best Cost: 17865


### Generowanie wyników

Za podstawowe dane przyjmujemy :

* population_size = 10 

* generations = 500

* crossover_rate = 0.6

* mutation_rate = 0.15

* selection_method=`tournament`
    
* crossover_method=`order`
    
* mutation_method=`swap`

## Algorytm zachłanny

In [108]:
## Klasyczny algorytm zachłanny 
def tsp_greedy(distance_matrix, start=0):
    n = len(distance_matrix)
    visited = [False] * n
    visited[start] = True
    path = [start]
    total_cost = 0
    current_city = start

    for _ in range(n - 1):
        # Znajdujemy najbliższe miasto którego nie odwiedziliśmy 
        next_city = min(
            (city for city in range(n) if not visited[city]),
            key=lambda city: distance_matrix[current_city][city]
        )
        # Dodajemy odległośc i zaznaczamy ze miasto jest już odwiedzone
        total_cost += distance_matrix[current_city][next_city]
        visited[next_city] = True
        path.append(next_city)
        current_city = next_city

    # Obliczanie końcowej trasy
    total_cost += distance_matrix[current_city][start]
    path.append(start)

    return path, total_cost

## Iteracja algorytmem zachłannym po każdym możliwym punkcie startowym 

def tsp_greedy_all_starts(distance_matrix):
    best_cost = float('inf')
    best_path = None

    for start in range(len(distance_matrix)):
        path, cost = tsp_greedy(distance_matrix, start=start)
        if cost < best_cost:
            best_cost = cost
            best_path = path
            point = start

    return best_cost, best_path, point

In [109]:

## Iteracja algorytmem zachłannym po każdym możliwym punkcie startowym 

def tsp_greedy_all_starts(distance_matrix):
    best_cost = float('inf')
    best_path = None

    for start in range(len(distance_matrix)):
        path, cost = tsp_greedy(distance_matrix, start=start)
        if cost < best_cost:
            best_cost = cost
            best_path = path
            point = start

    return best_cost, best_path, point

In [110]:

## Iteracja algorytmem zachłannym po każdym możliwym punkcie startowym 

def tsp_greedy_all_starts(distance_matrix):
    best_cost = float('inf')
    best_path = None

    for start in range(len(distance_matrix)):
        path, cost = tsp_greedy(distance_matrix, start=start)
        if cost < best_cost:
            best_cost = cost
            best_path = path
            point = start

    return best_cost, best_path, point

## Generowanie rozwiazań

In [111]:
# Klasyczny ( od punktu 1 )
classic_p_1, classic_c_1 = tsp_greedy(data1)
classic_p_2, classic_c_2 = tsp_greedy(data2)
classic_p_3, classic_c_3 = tsp_greedy(data3)

In [112]:
# Start we wszytskich punktach 
all_p_1, all_c_1, all_point_1 = tsp_greedy_all_starts(data1)
all_p_2, all_c_2, all_point_2 = tsp_greedy_all_starts(data2)
all_p_3, all_c_3, all_point_3 = tsp_greedy_all_starts(data3)

Zebranie wyników oraz zapis w pliku EXCEL 

In [ ]:
W1 =  {
    "Długość ścieżki": [classic_c_1,classic_c_2,classic_c_3 ]
}

p1 = pd.DataFrame(data = W1)
p1.index = ["Dane 1","Dane 2" , "Dane 3", ]

W2 =  {
    "Długość ścieżki": [all_p_1, all_p_2, all_p_3],
    "Punkt": [all_point_1,all_point_2, all_point_3]
}

p2 = pd.DataFrame(data = W2)
p2.index = ["Dane 1","Dane 2" , "Dane 3"]

resu = {
    "Klasyka": p1,
    "Wszystkie":p2    
}

# Ścieżka do pliku Excel
file_name = "Zachłanny.xlsx"

# Zapisujemy dane do Excela
with pd.ExcelWriter(file_name) as writer:
    for sheet_name, df in resu.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Wyniki zostały zapisane w pliku {file_name}")

Wyniki zostały zapisane w pliku Zachłanny.xlsx
